<a href="https://colab.research.google.com/github/Fr2zyRoom/ISLES2017_LesionSegmentation_Tutorial/blob/main/handling_ISLES2017_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ISLES2017 Lesion Segmentation Tutorial

In [ ]:
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1ItQhiEEKMC-xXwhcch-0qPJ7iw00IA0g"

In [ ]:
!unzip data.zip -d ./data

In [ ]:
!apt-get install tree

In [ ]:
from tqdm import tqdm
import os

import numpy as np
import pandas as pd
import seaborn as sns
import PIL.Image as Image

import nibabel as nib

import matplotlib.pyplot as plt

## About ISLES 2017 data
Lesion segmentation task: MRI DWI, ADC를 input data로 하여 lesion segmentation을 output으로 내는 challenge  
현대의 MR technique(diffusion/perfusion imaging)이 acutely infarcted tissue(“core”)와 hypo-perfused lesion tissue(“penumbra”)를 잘 구별할 수 있지만, 지금의 자동화 모델은 한계가 존재


## Opening
- Step 1. ISLES 2017 data specifications
- Step 2. nii 파일 불러오기: nibabel
    - Step 2-1. NIFTI header
    - Step 2-2. NIFTI image
    - Step 2-3. Ground truth
- Step 3. Pre-processing & Visualization
    - Step 3-1. Normalization
    - Step 3-2. Data cleansing
- Step 4. Save images to .npz

## Step 1. ISLES 2017 data specifications
데이터 구성은 DWI, ADC, perfusion map, OT - ground truth(lesion mask)로,  
각각의 디렉토리(폴더)에 nii 확장자로 존재

In [ ]:
!tree ./data/ISLES2017_Training/training_1/ -L 1

In [ ]:
train_df = pd.read_csv("./data/ISLES2017_Training.csv")
train_df.tail()

## Step 2. nii 파일 불러오기: nibabel
NIFTI 파일은 영상과 영상의 info가 저장된 header로 구성되어 있습니다.  
Python에서 NIFTI 파일은 nifti package를 통해 열어볼 수 있습니다.  
먼저 DWI 부터 열어봅니다.

In [ ]:
# 먼저 DWI 데이터를 open
sample_path = './data/ISLES2017_Training/training_1/VSD.Brain.XX.O.MR_4DPWI.127015/VSD.Brain.XX.O.MR_4DPWI.127015.nii'

In [ ]:
# call nii image (proxy)
ni_img = nib.load(sample_path)
ni_img

## Step 2-1. NIFTI header
nibabel.load를 통해 불러온 Nifti1Image에 .header를 붙여 header를 불러올 수 있습니다. 

In [ ]:
# call nii header
ni_header = ni_img.header
print(ni_header)

In [ ]:
# get method가 존재하여 원하는 header 정보를 불러올 수 있음.
ni_header.get('magic')

## Step 2-2. NIFTI image
nibabel.load를 통해 불러온 Nifti1Image에 .get_fdata()를 붙여 numpy 형태의 영상을 불러올 수 있습니다.

In [ ]:
ni_img_data = ni_img.get_fdata()
ni_img_data.shape #(Width, Height, Slices, 1, Acquisition Number)

In [ ]:
#The file is four dimensional. 
#The fourth dimension corresponds to the different diffusion orientation probed during the scan.
ni_img_data = np.squeeze(ni_img_data) # 1 제거
ni_img_data.shape

In [ ]:
#정제의 편의를 위해 전치
ni_img_data_tp = ni_img_data.transpose()
ni_img_data_tp.shape #(Acquisition Number, Slices, Height(row), Width(column))

In [ ]:
ni_img_data_sq20 = ni_img_data_tp[20]
ni_img_data_sq20.shape

In [ ]:
ni_img_data_sq20_slice11 = ni_img_data_sq20[10]
ni_img_data_sq20_slice11.shape

In [ ]:
plt.imshow(ni_img_data_sq20_slice11, cmap='gray')

## Step 2-3. Ground truth
ISLES 2017 dataset은 lesion mask 또한 NIFTI로 저장되어 있습니다.

In [ ]:
gt_path = './data/ISLES2017_Training/training_1/VSD.Brain.XX.O.OT.128050/VSD.Brain.XX.O.OT.128050.nii'

In [ ]:
gt_img = nib.load(gt_path)
gt_img_data = gt_img.get_fdata()
gt_img_data.shape #(Width, Height, Slices)

In [ ]:
gt_img_data_tp = gt_img_data.transpose()
gt_img_data_tp.shape #(Slices, Height(row), Width(column))

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(10,10))

ax[0].set_title('slice 11')
ax[0].imshow(ni_img_data_sq20_slice11, cmap='gray')

ax[1].set_title('slice 11 with mask')
ax[1].imshow(ni_img_data_sq20_slice11, cmap='gray')
ax[1].imshow(gt_img_data_tp[10], alpha=0.3)

## Step 3. Pre-processing & Visualization

In [ ]:
def w_sample_stack(stack, rows=3, cols=6, start_with=0, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[18,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/cols),int(i % cols)].set_title(f'slice {ind}')
        ax[int(i/cols),int(i % cols)].imshow(stack[ind], cmap='gray')
        ax[int(i/cols),int(i % cols)].axis('off')
    plt.show()

In [ ]:
w_sample_stack(ni_img_data_sq20) #이 경우 각 slice의 밝기 차이가 존재

In [ ]:
#정제의 편의를 위해 전치
ni_img_data_tp = ni_img_data.transpose()
ni_img_data_tp.shape #(Acquisition Number, Slices, Height(row), Width(column))

In [ ]:
ni_img_data_sq20 = ni_img_data_tp[20]
ni_img_data_sq20.shape

In [ ]:
# 다른 케이스
sample_path2 = './data/ISLES2017_Training/training_36/SMIR.Brain.XX.O.MR_4DPWI.188863/SMIR.Brain.XX.O.MR_4DPWI.188863.nii'
ni_img2 = nib.load(sample_path2)
ni_img_data2 = ni_img2.get_fdata()
ni_img_data2 = np.squeeze(ni_img_data2)
ni_img_data2 = ni_img_data2.transpose()
ni_img_data2_sq0 = ni_img_data2[0]
w_sample_stack(ni_img_data2_sq0) #이 경우 각 slice의 밝기 차이가 존재

In [ ]:
def sample_stack(stack, rows=3, cols=6, start_with=0, show_every=1, vmin=0, vmax=255):
    fig,ax = plt.subplots(rows,cols,figsize=[18,20])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/cols),int(i % cols)].set_title(f'slice {ind}')
        ax[int(i/cols),int(i % cols)].imshow(stack[ind], cmap='gray', vmin=vmin, vmax=vmax)
        ax[int(i/cols),int(i % cols)].axis('off')
    plt.show()

In [ ]:
print(ni_img_data2_sq0.min(), ni_img_data2_sq0.max())
sample_stack(ni_img_data2_sq0, vmin=ni_img_data2_sq0.min(), vmax=ni_img_data2_sq0.max()) 
#3D image에서의 min max값으로 고정을 해야 visualize가 잘됨. plt 특성

In [ ]:
plt.imshow(ni_img_data2_sq0[0], cmap='gray')

In [ ]:
# plot histogram
print(ni_img_data2_sq0[0].min(), ni_img_data2_sq0[0].max())
plt.figure(figsize=(10,5))
sns.histplot(ni_img_data2_sq0[0].flatten())
plt.show()

# plt.imshow는 영상의 최소 최대값으로 normalize하여 plot하기 때문에 intensity는 6이지만 밝게 보이는 것
# 따라서 plt.imshow의 parameter로 vmin, vmax값을 지정해줘야합니다.
# 주의: 실제 값은 상대적으로 작지만 plt.imshow로 볼 때만 밝게 보이는 것.

## Step 3-1. Normalization
모든 데이터 포인트가 동일한 정도의 스케일(중요도)로 반영되도록 해주는 게 정규화(Normalization)의 목표

Normalize 방법은 여럿 있지만 대표적으로 
1. Min-Max Normalization (최소-최대 정규화)  
(X - MIN) / (MAX-MIN) 
2. Z-Score Normalization (Z-점수 정규화)  

가 있습니다. 본 tutorial에서는 1번을 활용 예정

In [ ]:
# MR과 같은 3D image를 min-max normalize할 때는 각 slice마다 normalize하는 것이 아니라  
# 전체 slices에 대해 min-max normalize를 진행해야합니다.

In [ ]:
def normalize(img_arr):
    norm_arr = (img_arr-img_arr.min())/(img_arr.max()-img_arr.min())*255
    return norm_arr.astype(np.uint8)

In [ ]:
ni_img_data2_sq0_nrm = normalize(ni_img_data2_sq0)

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(ni_img_data2_sq0[0].flatten())
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(ni_img_data2_sq0_nrm[0].flatten())
plt.show()

In [ ]:
sample_stack(ni_img_data2_sq0_nrm, vmin=0, vmax=255)

## Step 3-2. Data cleansing
주어진 table 데이터와 영상 데이터의 매치가 잘 되는지 누락된 데이터는 없는지 확인합니다.

In [ ]:
train_df = pd.read_csv("./data/ISLES2017_Training.csv")
train_data_dir = './data/ISLES2017_Training'

In [ ]:
def get_subfolders_of(directory):
    """find subfolders in dir.
    
    Parameters:
        directory (str) -- folder directory
    
    Return:
        subfolder_ls (list) -- list of 'subfolders' in dir
    """
    subfolder_ls = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.is_dir():
                subfolder_ls.append(os.path.join(directory,entry.name))
    return subfolder_ls

In [ ]:
# subfolder 확인
train_subfolder_ls = [os.path.basename(p) for p in get_subfolders_of(train_data_dir)]
train_subfolder_ls = sorted(train_subfolder_ls)
len(train_subfolder_ls), train_subfolder_ls[5]

In [ ]:
# dataframe과 영상 데이터가 실제로 매치되는지 확인
train_df["DataExist"] = train_df["Case SMIR ID 1"].map(lambda x: 1 if x in train_subfolder_ls else 0)

In [ ]:
train_df.DataExist.value_counts()

In [ ]:
# 영상 데이터 없는 행 제거
train_df_clr = train_df[train_df.DataExist == 1].reset_index(drop=True)
len(train_df_clr)

In [ ]:
train_df_clr.to_csv("./data/ISLES2017_Training_clr.csv", index=False)

In [ ]:
pd.read_csv("./data/ISLES2017_Training_clr.csv")

In [ ]:
# mRS Score 분포
train_df_clr.MRSScore.value_counts().sort_values().plot(kind = 'barh')

## Step 4. Save images to .npz
모델 학습에 용이하도록 2d slice를 npz로 저장

In [ ]:
FILE_EXTENSION = ['.png', '.PNG', '.jpg', '.JPG', '.dcm', '.DCM', '.raw', '.RAW', '.svs', '.SVS']
IMG_EXTENSION = ['.png', '.PNG', '.jpg', '.JPG', '.jpeg', '.JPEG']
DCM_EXTENSION = ['.dcm', '.DCM']
RAW_EXTENSION = ['.raw', '.RAW']
NIFTI_EXTENSION = ['.nii']


def check_extension(filename, extension_ls=FILE_EXTENSION):
    return any(filename.endswith(extension) for extension in extension_ls)

def load_file_path(folder_path, extension_ls=FILE_EXTENSION, find_all=False):
    """find 'IMG_EXTENSION' file paths in folder.
    
    Parameters:
        folder_path (str) -- folder directory
        extension_ls (list) -- list of extensions
    
    Return:
        file_paths (list) -- list of 'extension_ls' file paths
    """
    
    file_paths = []
    assert os.path.isdir(folder_path), f'{folder_path} is not a valid directory'

    for root, _, fnames in sorted(os.walk(folder_path)):
        for fname in fnames:
            if check_extension(fname, extension_ls):
                path = os.path.join(root, fname)
                file_paths.append(path)
        if not find_all:
            break

    return file_paths[:]

In [ ]:
def gen_new_dir(new_dir):
    try: 
        if not os.path.exists(new_dir): 
            os.makedirs(new_dir) 
            #print(f"New directory!: {new_dir}")
    except OSError: 
        print("Error: Failed to create the directory.")

In [ ]:
def check_modality_in(folder_name, mod='4DPWI'):
    if folder_name.split('.')[-2].split('_')[-1] == mod:
        return True
    else:
        return False
    
def get_subfolders_of(directory):
    subfolder_ls = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.is_dir():
                subfolder_ls.append(os.path.join(directory,entry.name))
    return subfolder_ls

def find_modality_dir(data_dir, mod='4DPWI'):
    modality_ls = get_subfolders_of(data_dir)
    for modality in modality_ls:
        if check_modality_in(os.path.basename(modality), mod):
            return modality
    return None

def get_isles_data(dataset_dir, mod='4DPWI'):
    data_path_ls = []
    case_ls = get_subfolders_of(dataset_dir)
    for case_dir in case_ls:
        modality = find_modality_dir(case_dir, mod)
        if modality:
            data_path_ls.append(load_file_path(modality, NIFTI_EXTENSION)[0])
    return data_path_ls

def get_isles_data_with_gt(dataset_dir, mod='4DPWI'):
    data_path_ls = []
    case_ls = get_subfolders_of(dataset_dir)
    for case_dir in case_ls:
        modality = find_modality_dir(case_dir, mod)
        gt = find_modality_dir(case_dir, 'OT')
        if modality:
            data_path_ls.append([load_file_path(modality, NIFTI_EXTENSION)[0], 
                                 load_file_path(gt, NIFTI_EXTENSION)[0]])
    return data_path_ls

def normalize(img_arr):
    norm_arr = (img_arr-img_arr.min())/(img_arr.max()-img_arr.min())*255
    return norm_arr.astype(np.uint8)

In [ ]:
def save_isles_dwi_dataset_to_np(save_dir, dataset_dir):
    gen_new_dir(save_dir)
    isles_dataset = get_isles_data_with_gt(dataset_dir, mod='4DPWI')
    for data, gt in isles_dataset:
        fname = data.split('/')[-3]
        save_fname = os.path.join(save_dir, fname)
        gen_new_dir(save_fname)
        dwi_img_sq = np.squeeze(nib.load(data).get_fdata())
        gt_img = nib.load(gt).get_fdata()
        dwi_img_sq = dwi_img_sq.transpose(3,2,1,0)
        gt_img = gt_img.transpose(2,1,0).astype(np.uint8)
        for sq_idx, dwi_img in enumerate(dwi_img_sq):
            dwi_save_point = os.path.join(save_fname,'dwi',str(sq_idx).zfill(3))
            gt_save_point = os.path.join(save_fname,'mask',str(sq_idx).zfill(3))
            gen_new_dir(dwi_save_point)
            gen_new_dir(gt_save_point)
            for slice_idx in range(len(dwi_img)):
                np.save(os.path.join(dwi_save_point, str(slice_idx).zfill(3)+'.npy'), dwi_img[slice_idx])
                np.save(os.path.join(gt_save_point, str(slice_idx).zfill(3)+'.npy'), gt_img[slice_idx])
        print(f'Saved! {fname}(sq:{len(dwi_img_sq)} / slices:{len(dwi_img)})')

In [ ]:
def save_isles_dwi_dataset_to_png(save_dir, dataset_dir):
    gen_new_dir(save_dir)
    isles_dataset = get_isles_data_with_gt(dataset_dir, mod='4DPWI')
    for data, gt in isles_dataset:
        fname = data.split('/')[-3]
        save_fname = os.path.join(save_dir, fname)
        gen_new_dir(save_fname)
        dwi_img_sq = np.squeeze(nib.load(data).get_fdata())
        gt_img = nib.load(gt).get_fdata()
        dwi_img_sq = dwi_img_sq.transpose(3,2,1,0)
        gt_img = normalize(gt_img.transpose(2,1,0))
        for sq_idx, dwi_img in enumerate(dwi_img_sq):
            dwi_save_point = os.path.join(save_fname,'dwi',str(sq_idx).zfill(3))
            gt_save_point = os.path.join(save_fname,'mask',str(sq_idx).zfill(3))
            gen_new_dir(dwi_save_point)
            gen_new_dir(gt_save_point)
            dwi_img = normalize(dwi_img)
            for slice_idx in range(len(dwi_img)):
                Image.fromarray(dwi_img[slice_idx]).save(os.path.join(dwi_save_point, str(slice_idx).zfill(3)+'.png'))
                Image.fromarray(gt_img[slice_idx]).save(os.path.join(gt_save_point, str(slice_idx).zfill(3)+'.png'))
        print(f'Saved! {fname}(sq:{len(dwi_img_sq)} / slices:{len(dwi_img)})')

In [ ]:
def save_isles_dwi_dataset_to_np_norm(save_dir, dataset_dir):
    gen_new_dir(save_dir)
    isles_dataset = get_isles_data_with_gt(dataset_dir, mod='4DPWI')
    for data, gt in isles_dataset:
        fname = data.split('/')[-3]
        save_fname = os.path.join(save_dir, fname)
        gen_new_dir(save_fname)
        dwi_img_sq = np.squeeze(nib.load(data).get_fdata())
        gt_img = nib.load(gt).get_fdata()
        dwi_img_sq = dwi_img_sq.transpose(3,2,1,0)
        gt_img = gt_img.transpose(2,1,0).astype(np.uint8)
        for sq_idx, dwi_img in enumerate(dwi_img_sq):
            dwi_save_point = os.path.join(save_fname,'dwi',str(sq_idx).zfill(3))
            gt_save_point = os.path.join(save_fname,'mask',str(sq_idx).zfill(3))
            gen_new_dir(dwi_save_point)
            gen_new_dir(gt_save_point)
            dwi_img_norm = normalize(dwi_img)
            for slice_idx in range(len(dwi_img_norm)):
                np.save(os.path.join(dwi_save_point, str(slice_idx).zfill(3)+'.npy'), dwi_img_norm[slice_idx])
                np.save(os.path.join(gt_save_point, str(slice_idx).zfill(3)+'.npy'), gt_img[slice_idx])
        print(f'Saved! {fname}(sq:{len(dwi_img_sq)} / slices:{len(dwi_img_norm)})')

In [ ]:
def save_isles_adc_dataset_to_np(save_dir, dataset_dir):
    gen_new_dir(save_dir)
    isles_dataset = get_isles_data_with_gt(dataset_dir, mod='ADC')
    for data, gt in isles_dataset:
        fname = data.split('/')[-3]
        save_fname = os.path.join(save_dir, fname)
        gen_new_dir(save_fname)
        adc_img = np.squeeze(nib.load(data).get_fdata())
        gt_img = nib.load(gt).get_fdata()
        adc_img = adc_img.transpose(2,1,0)
        gt_img = gt_img.transpose(2,1,0).astype(np.uint8)
        adc_save_point = os.path.join(save_fname,'adc')
        gt_save_point = os.path.join(save_fname,'mask')
        gen_new_dir(adc_save_point)
        gen_new_dir(gt_save_point)
        for slice_idx in range(len(adc_img)):
            np.save(os.path.join(adc_save_point, str(slice_idx).zfill(3)+'.npy'), adc_img[slice_idx])
            np.save(os.path.join(gt_save_point, str(slice_idx).zfill(3)+'.npy'), gt_img[slice_idx])
        print(f'Saved! {fname}(slices:{len(adc_img)})')

In [ ]:
train_data_dir = './data/ISLES2017_Training'
data_path_ls = get_isles_data(train_data_dir, mod='4DPWI') #ADC
len(data_path_ls), data_path_ls[0]

In [ ]:
train_data_dir = './data/ISLES2017_Training'
save_dir = './data/ISLES2017_Training_2d_DWI'
save_isles_dwi_dataset_to_np(save_dir, train_data_dir)

In [ ]:
train_data_dir = './data/ISLES2017_Training'
save_dir = './data/ISLES2017_Training_2d_ADC'
save_isles_adc_dataset_to_np(save_dir, train_data_dir)